In [77]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from LSTM_utils import create_features_from_raw_df
from LSTM_utils import get_model_embeddings

import warnings

In [78]:
Y = "HIC"
batch_size = 64
epochs = 10
lr = 1e-3
random_seed = 42

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print("Using device:", device)

torch.manual_seed(random_seed)
np.random.seed(random_seed)

Using device: mps


## Prepping the DataFrame

In [79]:
csv_sequences = "../data/GDPa1_v1.2_sequences.csv" 
csv_properties = "../data/GDPa1_v1.2_20250814.csv"

seq_df = pd.read_csv(csv_sequences)
prop_df = pd.read_csv(csv_properties)

# target dataframe
target_df = prop_df[["antibody_id", Y]]

# merge target and sequences
seq_target = seq_df.merge(target_df, on="antibody_id", how="inner")
sequence_features = create_features_from_raw_df(seq_target)

# concat target and features
df = pd.concat(
    [seq_target.reset_index(drop=True),
     sequence_features.drop(columns=["antibody_id"]).reset_index(drop=True)],
    axis=1,
)

In [80]:
# transformer embeddings for sequences
df["seq_emb"] = get_model_embeddings(df)

100%|██████████| 16/16 [00:03<00:00,  4.66it/s]


## Feature Selection

In [92]:
nums = df.select_dtypes(include=[float, int]).columns

feature_corr = {}

for col in nums:
    if col != Y:
        rho, _ = spearmanr(df[col], df[Y])
        feature_corr[col] = rho

In [90]:
# strongest positive corr
pd.Series(feature_corr).sort_values(ascending=False).head(8)

vh_hydrophobic_count            0.267641
Y_vh_protein_sequence           0.238765
vh_aromatic_count               0.224281
vh_molar_extinction_reduced     0.210178
vh_molar_extinction_oxidized    0.210089
vh_aromaticity                  0.202629
vh_protein_sequence_length      0.198274
vl_hydrophobic_count            0.193455
dtype: float64

In [91]:
# strongest negative corr
pd.Series(feature_corr).sort_values(ascending=False).tail(5)

R_vh_protein_sequence   -0.169905
vl_pI                   -0.179104
vh_ph_7_45_charge       -0.226602
vh_ph_7_35_charge       -0.226898
vh_pI                   -0.231684
dtype: float64

In [84]:
# chose the features with highest corr (>0.18)
engineered_cols = [
    "vh_hydrophobic_count",
    "Y_vh_protein_sequence",
    "vh_aromatic_count",
    "vh_molar_extinction_oxidized",
    "vh_molar_extinction_reduced",
    "vh_aromaticity",
    "vh_protein_sequence_length",
    "vl_hydrophobic_count",
    "vh_pI",
    "vh_ph_7_35_charge",
    "vh_ph_7_45_charge"
]

df = df.dropna(subset=[Y]).reset_index(drop=True)


## Dataset Contruction

In [ ]:
class AntibodySeqDataset(Dataset):
    def __init__(self, df, feature_cols=None):
        self.seq_embs = df["seq_emb"].to_list()
        self.y = df[Y].astype(float).values
        if feature_cols:
            self.features = df[feature_cols].astype(np.float32).values
        else:
            self.features = None

    def __len__(self):
        return len(self.y)

    # what fed into model
    def __getitem__(self, idx):
        seq = torch.tensor(self.seq_embs[idx], dtype=torch.float32)
        target = torch.tensor(self.y[idx], dtype=torch.float32)

        sample = {"seq": seq, "target": target}

        if self.features is not None:
            sample["features"] = torch.tensor(self.features[idx], dtype=torch.float32)

        return sample
    

def collate_fn(batch):
    seqs = [b["seq"] for b in batch]
    lengths = torch.tensor([s.size(0) for s in seqs], dtype=torch.long)

    # pad
    padded = nn.utils.rnn.pad_sequence(
        seqs, batch_first=True, padding_value=0.0
    )

    batch_out = {
        "seq": padded,
        "seq_lengths": lengths,
        "target": torch.stack([b["target"] for b in batch]),
    }

    if "features" in batch[0]:
        batch_out["features"] = torch.stack([b["features"] for b in batch])

    return batch_out

## Model Architecture

In [ ]:
class AntibodyLSTMModel(nn.Module):
    def __init__(self,
                 input_size=20,
                 hidden_size=64,
                 num_layers=1,
                 dropout=0.1,
                 engineered_feat_dim=0):
        super().__init__()

        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
        )

        input_dim = hidden_size + engineered_feat_dim

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, seq, seq_lengths, features=None):

        packed = nn.utils.rnn.pack_padded_sequence(
            seq, seq_lengths.cpu(), batch_first=True, enforce_sorted=False
        )

        output, (hidden, cell) = self.lstm(packed)
        seq_repr = hidden[-1] 

        combined = seq_repr
        if features is not None:
            combined = torch.cat([combined, features], dim=1)

        out = self.mlp(combined).squeeze(-1)
        return out

## Training

In [87]:
def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()

    total_loss = 0.0
    all_preds, all_targets = [], []

    for batch in loader:
        seq = batch["seq"].to(device)
        seq_lengths = batch["seq_lengths"].to(device)
        targets = batch["target"].to(device)

        feats = batch.get("features")
        if feats is not None:
            feats = feats.to(device)

        if train:
            optimizer.zero_grad()

        with torch.set_grad_enabled(train):
            preds = model(seq, seq_lengths, features=feats)
            loss = criterion(preds, targets)

            if train:
                loss.backward()
                optimizer.step()

        total_loss += loss.item() * targets.size(0)
        all_preds.append(preds.detach().cpu().numpy())
        all_targets.append(targets.detach().cpu().numpy())

    avg_loss = total_loss / len(loader.dataset)
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    if np.std(all_preds) < 1e-8 or np.std(all_targets) < 1e-8:
        spearman = np.nan
    else:
        rho, _ = spearmanr(all_targets, all_preds)
        spearman = float(rho)

    return avg_loss, spearman

In [88]:
for test_fold in range(5):

    
    train_df, test_df = df.loc[df['hierarchical_cluster_IgG_isotype_stratified_fold']!=test_fold], df.loc[df['hierarchical_cluster_IgG_isotype_stratified_fold']==test_fold]

    train_df.loc[:, engineered_cols] = train_df[engineered_cols].astype(np.float64)
    test_df.loc[:, engineered_cols] = test_df[engineered_cols].astype(np.float64)

    # normalize all the other features
    scaler = StandardScaler()

    # implemented these warning catches because pandas was being really picky
    # even though I assigned float64 to literally everything
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=".*incompatible dtype.*")
        train_df.loc[:, engineered_cols] = scaler.fit_transform(train_df[engineered_cols])

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message=".*incompatible dtype.*")
        test_df.loc[:, engineered_cols]  = scaler.transform(test_df[engineered_cols])

    train_dataset = AntibodySeqDataset(train_df, feature_cols=engineered_cols)
    test_dataset  = AntibodySeqDataset(test_df,  feature_cols=engineered_cols)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              shuffle=True,  collate_fn=collate_fn)
    test_loader  = DataLoader(test_dataset,  batch_size=batch_size,
                              shuffle=False, collate_fn=collate_fn)

    emb_dim = train_df["seq_emb"].iloc[0].shape[1]

    model = AntibodyLSTMModel(
        input_size=emb_dim,
        hidden_size=64,
        num_layers=1,
        dropout=0.1,
        engineered_feat_dim=len(engineered_cols)).to(device)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(1, epochs + 1):
        train_loss, train_rho = run_epoch(train_loader, train=True)
        val_loss, val_rho = run_epoch(test_loader, train=False)
        print(
            f"Epoch {epoch:02d} : "
            f"train ρ: {train_rho:.3f} | "
            f"val ρ: {val_rho:.3f}"
        )
    print(f"Finished running Fold #{test_fold}")

Epoch 01 : train ρ: 0.049 | val ρ: 0.221
Epoch 02 : train ρ: 0.077 | val ρ: 0.263
Epoch 03 : train ρ: 0.066 | val ρ: 0.302
Epoch 04 : train ρ: 0.090 | val ρ: 0.285
Epoch 05 : train ρ: 0.053 | val ρ: 0.313
Epoch 06 : train ρ: 0.194 | val ρ: 0.349
Epoch 07 : train ρ: 0.164 | val ρ: 0.349
Epoch 08 : train ρ: 0.185 | val ρ: 0.353
Epoch 09 : train ρ: 0.306 | val ρ: 0.334
Epoch 10 : train ρ: 0.337 | val ρ: 0.340
Finished running Fold #0
Epoch 01 : train ρ: 0.106 | val ρ: 0.260
Epoch 02 : train ρ: 0.029 | val ρ: 0.161
Epoch 03 : train ρ: 0.011 | val ρ: 0.183
Epoch 04 : train ρ: 0.016 | val ρ: 0.181
Epoch 05 : train ρ: 0.005 | val ρ: 0.197
Epoch 06 : train ρ: 0.035 | val ρ: 0.218
Epoch 07 : train ρ: 0.076 | val ρ: 0.234
Epoch 08 : train ρ: 0.109 | val ρ: 0.213
Epoch 09 : train ρ: 0.177 | val ρ: 0.232
Epoch 10 : train ρ: 0.223 | val ρ: 0.239
Finished running Fold #1
Epoch 01 : train ρ: -0.127 | val ρ: -0.347
Epoch 02 : train ρ: -0.133 | val ρ: -0.387
Epoch 03 : train ρ: -0.148 | val ρ: -0.388
E